In [202]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt

In [203]:
df = pd.read_csv("Data/Data.csv")
df = df[["country", "commodity", "loss_percentage", "activity", "food_supply_stage"]]
df

,country,commodity,loss_percentage,activity,food_supply_stage
0,Myanmar,"Groundnuts, excluding shelled",5.22,NaN,Whole supply chain
1,Myanmar,"Groundnuts, excluding shelled",5.43,NaN,Whole supply chain
2,Myanmar,"Groundnuts, excluding shelled",5.61,NaN,Whole supply chain
3,Myanmar,"Groundnuts, excluding shelled",5.40,NaN,Whole supply chain
4,Myanmar,"Groundnuts, excluding shelled",5.00,NaN,Whole supply chain
...,...,...,...,...,...
27768,Zambia,Millet,1.27,Storage,Farm
27769,Zambia,Millet,2.50,Winnowing,Farm
27770,Zambia,Millet,1.00,Transportation,Transport
27771,Zambia,Millet,2.38,Storage,Storage


In [204]:
# df[df.food_supply_stage=="Whole supply chain"]

Missing values mostly belong to the rows whose food_supply_stage refer to 'Whole supply chain'.

In [205]:
df=df.dropna(axis=0).reset_index().drop(["index"],axis=1)

In [206]:
# def prep(df):
#     X=df[["country","commodity","food_supply_stage","activity"]]
#     splitted=X['activity'].str.split(',', expand=True)
#     X=pd.concat([X,splitted],axis=1)
#     X.drop(["activity"],axis=1,inplace=True)
#     listt=[]
#     for col in range(0,11):
#         listt.append(set(X[col].unique()))
#     listtt=[]
#     for i in range(0,11):
#         for item in listt[i]:
#             if item==None:
#                 continue
#             listtt.append(item.strip())
#     listtt=list(set(listtt))
#     listtt.sort()

#     zeros=np.zeros([X.shape[0],len(listtt)],int)
#     activity_cols=pd.DataFrame(zeros,columns=listtt)

#     X=X.applymap(lambda x: x.split()[0] if x!=None else x )

#     dummies_0=pd.get_dummies(X[0],columns=listtt)
#     dummies_1=pd.get_dummies(X[1],columns=listtt)
#     dummies_2=pd.get_dummies(X[2],columns=listtt)
#     dummies_3=pd.get_dummies(X[3],columns=listtt)
#     dummies_4=pd.get_dummies(X[4],columns=listtt)
#     dummies_5=pd.get_dummies(X[5],columns=listtt)
#     dummies_6=pd.get_dummies(X[6],columns=listtt)
#     dummies_7=pd.get_dummies(X[7],columns=listtt)
#     dummies_8=pd.get_dummies(X[8],columns=listtt)
#     dummies_9=pd.get_dummies(X[9],columns=listtt)
#     dummies_10=pd.get_dummies(X[10],columns=listtt)

#     arr=[dummies_0,dummies_1,dummies_2,dummies_3,dummies_4,dummies_5,dummies_6,dummies_7,dummies_8,dummies_9,dummies_10]
#     for i in arr:
#         for col in i.columns:
#             activity_cols[col]=activity_cols[col].where(activity_cols[col]>i[col],i[col])
    
#     X=pd.concat([X,activity_cols],axis=1)

#     X.drop([0,1,2,3,4,5,6,7,8,9,10],axis=1,inplace=True)

#     dummies=pd.get_dummies((X[["country","commodity","food_supply_stage"]]))
#     X=pd.concat([X,dummies],axis=1)
#     X.drop(["country","commodity","food_supply_stage"],axis=1,inplace=True)

#     return X

In [207]:
X=df[["country","commodity","food_supply_stage","activity"]]
Y=df.loss_percentage

In [208]:
splitted=X['activity'].str.split(',', expand=True)

In [209]:
X=pd.concat([X,splitted],axis=1)

In [210]:
X.drop(["activity"],axis=1,inplace=True)

In [211]:
listt=[]
for col in range(0,11):
    listt.append(set(X[col].unique()))

listtt=[]
for i in range(0,11):
    for item in listt[i]:
        if item==None:
            continue
        listtt.append(item.strip())
listtt=list(set(listtt))
listtt.sort()

zeros=np.zeros([X.shape[0],len(listtt)],int)
activity_cols=pd.DataFrame(zeros,columns=listtt)
activity_cols

,Assembling,Bagging,Blanching,Bundling,Cleaning,Collection,Consumption,Curing,Dewatering,Distribution,...,Sorting,Stacking,Storage,Threshing,Trading,Transportation,Unloading,Washing,Wholesale,Winnowing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
X=X.applymap(lambda x: x.split()[0] if x!=None else x)
dummies_0=pd.get_dummies(X[0],columns=listtt)
dummies_1=pd.get_dummies(X[1],columns=listtt)
dummies_2=pd.get_dummies(X[2],columns=listtt)
dummies_3=pd.get_dummies(X[3],columns=listtt)
dummies_4=pd.get_dummies(X[4],columns=listtt)
dummies_5=pd.get_dummies(X[5],columns=listtt)
dummies_6=pd.get_dummies(X[6],columns=listtt)
dummies_7=pd.get_dummies(X[7],columns=listtt)
dummies_8=pd.get_dummies(X[8],columns=listtt)
dummies_9=pd.get_dummies(X[9],columns=listtt)
dummies_10=pd.get_dummies(X[10],columns=listtt)

arr=[dummies_0,dummies_1,dummies_2,dummies_3,dummies_4,dummies_5,dummies_6,dummies_7,dummies_8,dummies_9,dummies_10]
for i in arr:
    for col in i.columns:
        activity_cols[col]=activity_cols[col].where(activity_cols[col]>i[col],i[col])

X=pd.concat([X,activity_cols],axis=1)
X.drop([0,1,2,3,4,5,6,7,8,9,10],axis=1,inplace=True)

dummies=pd.get_dummies((X[["country","commodity","food_supply_stage"]]))
X=pd.concat([X,dummies],axis=1)
X.drop(["country","commodity","food_supply_stage"],axis=1,inplace=True)
X

,Assembling,Bagging,Blanching,Bundling,Cleaning,Collection,Consumption,Curing,Dewatering,Distribution,...,food_supply_stage_Market,food_supply_stage_Packing,food_supply_stage_Post-harvest,food_supply_stage_Processing,food_supply_stage_Retail,food_supply_stage_Storage,food_supply_stage_Trader,food_supply_stage_Transport,food_supply_stage_Whole,food_supply_stage_Wholesale
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
20868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [213]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=30)
X_train, X_val, Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)

In [214]:
model=MLPRegressor().fit(X_train, Y_train)
y_pred=model.predict(X_val)
print("MSE (Before Tuning)=",np.sqrt(mean_squared_error(Y_val,y_pred)))

# "hidden_layer_sizes":[(10,2),(10,20),(10,100),(5,10)]

# MODEL TUNING***
# mlp_params={"alpha":[10,1, 0.1],
#              "hidden_layer_sizes":[(10,20),(50,50),(10,10)],
#              "learning_rate_init":[0.001,0.01,0.1,1]}
# mlp_cv_model=GridSearchCV(model,mlp_params,cv=5, verbose=2, n_jobs=-1).fit(X_train,Y_train)
# print(mlp_cv_model.best_params_)

# {'alpha': 1, 'hidden_layer_sizes': (10, 10), 'learning_rate_init': 0.001}
# 'alpha': 10, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.001}

# #final modeli oluşturma
mlp_tuned=MLPRegressor(alpha=10, hidden_layer_sizes=(50,50), learning_rate_init=0.001).fit(X_train,Y_train)
#y_pred=mlp_tuned.predict(X_val)
y_pred=mlp_tuned.predict(X_val)  #x_val vardı/////////////////////////////////////////////////////////////////////////////////////////////////////
#y_pred=pd.DataFrame(data=y_pred, columns=["y_pred"])
print("MSE=",np.sqrt(mean_squared_error(Y_val,y_pred)))
    
print("MAE=",mean_absolute_error(Y_val,y_pred))
y_pred_ann=mlp_tuned.predict(X_val)
print(mlp_tuned.score(X_val, Y_val))

/Users/murathansaygili/Desktop/SIC/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MSE (Before Tuning)= 2.71170677116655
MSE= 2.5981063852463975
MAE= 0.8110397627318872
0.6031334049455294


In [215]:
test_pred=mlp_tuned.predict(X_test)
MSE = np.square(np.subtract(Y_test,test_pred)).mean() 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

2.6740773764684027


In [216]:
mean_absolute_error(Y_test,test_pred)

0.8396976881488085